In [ ]:
import pandas as pd

In [405]:
""" Get stuff out of Netfile v2 API
"""
from pprint import PrettyPrinter
from pathlib import Path
import os
import requests

BASE_URL = 'https://netfile.com/api/campaign'
CONTRIBUTION_FORM = 'F460A'
EXPENDITURE_FORM = 'F460E'

PARAMS = { 'aid': 'COAK' }

def get_auth_from_env_file(filename: str='.env'):
    """ Split .env file on newline and look for API_KEY and API_SECRET
        Return their values as a tuple
    """
    env_file=Path(filename)
    auth_keys = [ 'API_KEY', 'API_SECRET' ]
    if env_file.exists():
        auth = tuple( v for _, v in sorted([
            ln.split('=') for ln in
            env_file.read_text(encoding='utf8').strip().split('\n')
            if ln.startswith(auth_keys[0]) or ln.startswith(auth_keys[1])
        ], key=lambda ln: auth_keys.index(ln[0])))
    else:
        auth=tuple(os.environ[key] for key in auth_keys)
            
    return auth

AUTH=get_auth_from_env_file()

pp = PrettyPrinter()

def get_filing(offset=0):
    """ Get a filing
    """
    url = f'{BASE_URL}/filing/v101/filings'

    params = { **PARAMS }
    if offset > 0:
        params['offset'] = offset

    res = requests.get(url, params=params, auth=AUTH)
    body = res.json()
    results = body.pop('results')

    return results, body
def get_filer(filer_nid):
    """ Get one filer
    """
    url = f'{BASE_URL}/filer/v101/filers'

    res = requests.get(url, params={ **PARAMS, 'filerNid': filer_nid }, auth=AUTH)
    body = res.json()

    return body['results']
def list_filers():
    """ Get all the elections
    """
    url = f'{BASE_URL}/filer/v101/filers?Limit=100000'

    res = requests.get(url, params=PARAMS, auth=AUTH)
    body = res.json()

    return body['results']
def list_elections_influences(id):
    """ Get all the elections
    """
    url = f'{BASE_URL}/election/v101/election-influences?Limit=100000&ElectionNid={id}'

    res = requests.get(url, params=PARAMS, auth=AUTH)
    body = res.json()

    return body['results']
def list_elections():
    """ Get all the elections
    """
    url = f'{BASE_URL}/election/v101/elections?Limit=100000'

    res = requests.get(url, params=PARAMS, auth=AUTH)
    body = res.json()

    return body['results']

In [ ]:
filing=get_filing()

In [ ]:
filers_response=list_filers()

In [ ]:
elections=list_elections()

In [ ]:
fppc={'filerNid':[], 'fppc_id':[]}
for item in filers_response:
    fppc['filerNid'].append(item.get('filerNid',{}))
    fppc['fppc_id'].append(item.get('registrations',{}).get('CA SOS',None))
fppc_df=pd.DataFrame(fppc)

In [ ]:
files=filing[0]
row=[]
failed=[]
filer_ids=[]
filings={'form_type':[],'fppc_id':[],'filer_id':[],'filer_name':[],'candidate_name':[],'filing_id':[],'filing_date':[],'electionNid':[],'types':[]}
for file in files:
    
    form_type=file.get('specificationRef',{}).get('name', 'None')

    filer_name=file.get('filerMeta',{}).get('commonName', 'None')

    filer_id=file.get('filerMeta',{}).get('filerId', 'None')
    filer_ids.append(filer_id)
    
    filing_id=file.get('filingMeta',{}).get('issuedFilingId', 'None')

    filing_date=file.get('filingMeta',{}).get('legalFilingDate', 'None')

    fppc_id=file.get('filerMeta',{}).get('strings',{}).get('Registration_CA SOS', 'None')

    filings['form_type'].append(form_type)
    filings['filer_name'].append(filer_name)
    filings['filer_id'].append(filer_id)
    filings['filing_id'].append(filing_id)
    filings['filing_date'].append(filing_date)
    filings['fppc_id'].append(fppc_id)


In [ ]:
Candidate_or_Officeholder=[item for item in filers_response if item['committeeTypes']==['Candidate or Officeholder']]
Conflict_of_Interest=[item for item in filers_response if item['committeeTypes']==['Conflict of Interest']]
General_Purpose=[item for item in filers_response if item['committeeTypes']==['General Purpose']]
Independent_Expenditure=[item for item in filers_response if item['committeeTypes']==['Independent Expenditure']]
Major_Donor=[item for item in filers_response if item['committeeTypes']==['Major Donor']]
Person=[item for item in filers_response if item['committeeTypes']==['Person']]
Primarily_Formed_Candidate=[item for item in filers_response if item['committeeTypes']==['Primarily Formed Candidate']]
Primarily_Formed_Measure=[item for item in filers_response if item['committeeTypes']==['Primarily Formed Measure']]

In [ ]:
filers={'candidateName':[],'isTerminated':[],'committeeTypes':[],'committeeName':[],'fppc_id':[]}
for candidate in Candidate_or_Officeholder:
    filers['fppc_id'].append(candidate.get('registrations',{}).get('CA SOS','None'))
    filers['candidateName'].append(candidate.get('candidateName', 'None'))
    filers['committeeName'].append(candidate.get('filerName', 'None'))
    filers['isTerminated'].append(candidate.get('isTerminated', 'None'))
    filers['committeeTypes'].append(candidate.get('committeeTypes', 'None')[0])


In [ ]:
election_list=[]
previous_df=pd.DataFrame()
for election in elections:
    candidates=election['candidates']
    seats=election['seats']
    election_name=election['electionCaption']
    electionNid=election['electionNid']
    election_key={'election_name':election_name, 'electionNid':electionNid}
    election_list.append(election_key)
    if candidates and seats:
        seat_df=pd.DataFrame(seats)
        candidate_df=pd.DataFrame(candidates)
        merge_df=candidate_df.merge(seat_df, on='seatNid')
        current_df=merge_df[['candidateNid','candidateName','seatNid','officeName','electionNid']]
        previous_df=pd.concat([previous_df,current_df],ignore_index=True)
    #merge_df=merge_df.merge(election_key, on='electionNid')
election_df=pd.DataFrame(election_list)
final_df=previous_df.merge(election_df, on='electionNid')

In [ ]:
election_ids=list(set(final_df['electionNid'].to_list()))

In [ ]:
previous_df=pd.DataFrame()
for id in election_ids:
    influences=list_elections_influences(id)
    influences_dic={'filerNid': [],'electionNid': [],'seatNid': [],'candidateNid': [],'committeeName':[],'election_name': []} # ,'isWinner':[],'isIncumbent':[]
    for candidate in influences:   
        influences_dic['filerNid'].append(candidate.get('filerNid', 'None'))
        influences_dic['election_name'].append(candidate.get('electionCaption', 'None'))
        influences_dic['committeeName'].append(candidate.get('committeeName', 'None'))
        influences_dic['electionNid'].append(candidate.get('electionNid', 'None'))
        influences_dic['seatNid'].append(candidate.get('seatNid', 'None'))
        influences_dic['candidateNid'].append(candidate.get('candidateNid', 'None'))
        # influences_dic['isWinner'].append(candidate.get('candidate',{}).get('isWinner', 'None'))
        # influences_dic['isIncumbent'].append(candidate.get('candidate',{}).get('isIncumbent', 'None'))
        current_df=pd.DataFrame(influences_dic)
        current_df=current_df
        previous_df=pd.concat([previous_df,current_df],ignore_index=True)
df3=previous_df


In [ ]:
dfNew = final_df.merge(df3, on=['candidateNid','election_name','electionNid','seatNid'])

In [ ]:
core_df=dfNew[['candidateName','officeName','committeeName','election_name','filerNid']]

In [ ]:
dfdf=core_df.merge(fppc_df, on=['filerNid'])

In [ ]:
dfdf.drop_duplicates().head(60)